# Azure Data Explorer Condition-based Alerting

> POC: Designed to read incoming data (via an Azure Function) every 5 seconds and run a series of pre-defined rules against it. If the condition matched, and a record is part of the result set for that alert query, an alert will be written into the AlertOutput table.

In [ ]:
# Install Python dependencies
%pip install azure-kusto-data pandas

In [116]:
# Import dependencies
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.helpers import dataframe_from_result_table
from datetime import datetime
import uuid

In [99]:
# Connect to ADX
cluster = ""
client_id = ""
client_secret = ""
authority_id = ""

kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(cluster, client_id, client_secret, authority_id)

# Kusto Client
Client = KustoClient(kcsb)
database = ""

In [143]:
# Get the rules we want to evaluate
get_rules_query = "AlertRules | where RuleId == '42889516-6361-46e5-a0f5-a88a70749294'"
response = Client.execute(database, get_rules_query)
rules = dataframe_from_result_table(response.primary_results[0]) # This dataframe will contain the rules to assess
#print(rules)

In [147]:
# Alerts to be written to the Alerts table
alerts = []

# Evaluate the rules
for index, rule in rules.iterrows():
    # For each rule..
    print(f"Assessing rule: {rule['AlertName']}")
    print(f"Query to run: '{rule['Query']}'")

    # Run the query defined in this rule
    response = Client.execute(database, rule['Query'])
    # Will contain the result set where the condition was true
    result_set = dataframe_from_result_table(response.primary_results[0])

    if (result_set.empty):
        print('Alert condition not met, moving on to the next rule..')
    else:
        print('The alert condition was met. Generating Alert..')
        print(result_set)
        # TODO: Do something now that we know the condition is true.
    
        # Output: {id}, {rule_id}, {rule_name}, {importance}, {email_to}, {generated_at}
        out_alert = f"'{uuid.uuid4()}', '{rule['RuleId']}', '{rule['AlertName']}', '{rule['Importance']}', '{rule['AlertOwner']}'"
        # Append to alerts so that we can write it to the table later
        alerts.append(out_alert)

# Write the alert to the table
if (len(alerts) > 0):
    # TODO: Write to Alerts table

    # write_alerts_query = f".set-or-append async AlertOutput <| print {alerts[0]}"
    # Client.execute(database, write_alerts_query)


Assessing rule: Temp between 20-25 Degrees
Query to run: 'CuratedTelemetry | count'
The alert condition was met. Generating Alert..
     Count
0  1004409
